In [1]:
import requests
import sys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
data = []

url_prefix = 'https://www.tunefind.com/show/peaky-blinders/season-'
for i in range(1, 6):
    resp_season = requests.get(url_prefix + str(i), timeout=60)
    html_season = BeautifulSoup(resp_season.content)
    episodes = html_season.find_all("div", {"class": "EpisodeListItem__container___3A-mL"})
    for episode in episodes:
        tmp0 = episode.find('a')
        tmp1 = tmp0.text.split('·')
        episode_name = tmp1[0].strip()+tmp1[1].strip()
        url = "https://www.tunefind.com"+tmp0['href']
        resp_episode = requests.get(url, timeout=60)
        html_episode = BeautifulSoup(resp_episode.content)
        songs = html_episode.find_all("div", {"class": "SongRow__center___1HKjk"})
        for song in songs:
            try:
                _title = song.find("a", {"class": "SongTitle__link___2OQHD"})
                _artist = song.find("a", {"class": "Subtitle__subtitle___1rSyh"})
                d = {'season': i, 'episode': episode_name, 
                    'song_title': _title.text.strip(), 'song_url': "https://www.tunefind.com"+_title['href'],
                    'artist_name': _artist.text.strip(), 'artist_url': "https://www.tunefind.com"+_artist['href']
                    }
                data.append(d)
            except:
                pass

In [3]:
colnames = ['season', 'episode', 'song_title', 'song_url', 'artist_name', 'artist_url']
df_songs = []
for d in data:
    row = [d[k] for k in colnames]
    df_songs.append(row)
df_songs = pd.DataFrame(df_songs, columns=colnames)

In [4]:
df_songs.to_csv('./df_songs.csv', index=False)

In [5]:
df_songs['artist_name'].value_counts().head(20)

PJ Harvey                      24
Nick Cave & The Bad Seeds      20
Anna Calvi                     12
Arctic Monkeys                 12
The White Stripes              10
Radiohead                       8
Mearl                           8
Dickon Hinchliffe               8
The Kills                       5
Nick Cave & Warren Ellis        5
Tom Waits                       5
Laura Marling                   4
The Last Shadow Puppets         4
IDLES                           4
The Raconteurs                  3
Antony Genn                     3
Royal Blood                     3
Johnny Cash                     2
Black Sabbath                   2
Black Rebel Motorcycle Club     2
Name: artist_name, dtype: int64

In [6]:
df_tmp = df_songs.groupby('season')['artist_name'].value_counts()
df_top_artists = []
colnames = []
for i in range(1, 6):
    df_top_artists.append(df_tmp.loc[i].head(10).index)
    colnames.append('S'+str(i)+'_artist')
    df_top_artists.append(df_tmp.loc[i].head(10).values)
    colnames.append('S'+str(i)+'_count')
df_top_artists = pd.DataFrame(np.array(df_top_artists).transpose(),
                              columns=colnames)
    

In [7]:
df_top_artists

,S1_artist,S1_count,S2_artist,S2_count,S3_artist,S3_count,S4_artist,S4_count,S5_artist,S5_count
0,Nick Cave & The Bad Seeds,10,PJ Harvey,21,Dickon Hinchliffe,8,Antony Genn,3,Anna Calvi,12
1,The White Stripes,10,Arctic Monkeys,9,Radiohead,4,Laura Marling,3,IDLES,3
2,Mearl,7,Nick Cave & The Bad Seeds,4,The Last Shadow Puppets,4,Nick Cave & The Bad Seeds,3,Black Sabbath,2
3,Nick Cave & Warren Ellis,5,Royal Blood,2,Arctic Monkeys,3,Black Rebel Motorcycle Club,1,Radiohead,2
4,The Raconteurs,3,The Kills,2,Nick Cave & The Bad Seeds,3,Dara MacMahon,1,Black Strobe,1
5,Tom Waits,3,APM International Orchestra,1,PJ Harvey,3,FIDLAR,1,Cabbage,1
6,Jack White,2,Ane Brun,1,Leonard Cohen,2,Foals,1,Hotel Lux,1
7,Alberto Albertini,1,Black Rebel Motorcycle Club,1,The Kills,2,Frank Carter & The Rattlesnakes,1,Jehnny Beth,1
8,Dan Auerbach,1,Dan Auerbach,1,Tom Waits,2,Gene Austin,1,Joy Division,1
9,Firehouse Five Plus Two,1,Grinderman,1,Archie Bronson Outfit,1,IDLES,1,Mari Samuelson,1
